In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm

$min_y (f'(x_k)^T (y-x_k)+1/2 * (y-x_k)^T f''(x_k) (y-x_k) +L/6 ||y-x_k||^3)$

In [19]:
def to_vertical_matrix(vector):
    return tf.expand_dims(vector, 1)

# N = 2
# x_k_value = [2.0, 1.0]
# L = 6.0
# point = (0,0)

# Graph: https://www.desmos.com/calculator/9pd0mtvbdw
N = 1
x_k_value = (2.0,)
L = 6.0
point = (0,)

step_factor = 0.001

x_k = tf.placeholder(tf.float64, [N], "x_k")
y = tf.placeholder(tf.float64, [N], "y")

# f = x_k[0] ** 2 + x_k[1] ** 2
f = x_k[0] ** 2

g_f = tf.gradients(f, x_k)[0]
h_f = tf.hessians(f, x_k)[0]

target = tf.transpose(to_vertical_matrix(g_f)) @ to_vertical_matrix(y - x_k) + \
    0.5 * tf.transpose(to_vertical_matrix(y - x_k)) @ h_f @ to_vertical_matrix(y - x_k) + \
    L / 6 * (tf.norm(y - x_k) ** 3)


In [20]:
maximum_steps = 50_000  # Make it finish

answer = pd.DataFrame(columns=[
    "Step",
    "target",
    "y",
])

g_target = tf.gradients(target, y)
with tf.Session() as sess:
    previous_value = sess.run(target, feed_dict={x_k: x_k_value, y: point})[0][0]

    answer.loc[len(answer)] = [0, previous_value, point]

    for steps in tqdm(range(1, maximum_steps + 1)):
        gradient = sess.run(g_target, feed_dict={x_k: x_k_value, y: point})[0]
        point = tuple(np.array(point) - step_factor * np.array(gradient))
        current_value = sess.run(target, feed_dict={x_k: x_k_value, y: point})[0][0]

        answer.loc[len(answer)] = [steps, current_value, point]

        if abs(current_value - previous_value) < 1e-6:
            break

        previous_value = current_value

answer

  1%|▏         | 714/50000 [00:03<04:23, 187.04it/s]


,Step,target,y
0,0,4.000000,"(0,)"
1,1,3.857006,"(0.012,)"
2,2,3.717973,"(0.023832432,)"
3,3,3.582771,"(0.0355004819064451,)"
4,4,3.451277,"(0.04700725601240164,)"
...,...,...,...
711,711,-2.064533,"(1.1270060142725586,)"
712,712,-2.064534,"(1.1270383577413623,)"
713,713,-2.064535,"(1.1270704671124443,)"
714,714,-2.064536,"(1.1271023440863812,)"
